In [172]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
tickers = yf.Tickers('msft aapl goog')

# access each ticker using (example)
tickers.tickers['MSFT'].info
tickers.tickers['AAPL'].history(period="1mo")
tickers.tickers['GOOG'].actions

,Dividends,Stock Splits
Date,,
2014-03-27,0.0,2.002000
2015-04-27,0.0,1.002746
2022-07-18,0.0,20.000000


In [3]:
yf.download(tickers = "SPY AAPPPLL",  # list of tickers
            period = "1y",         # time period
            interval = "1d",       # trading interval
            ignore_tz = True,      # ignore timezone when aligning data from different exchanges?
            prepost = False )      # download pre/post market hours data?

[*********************100%***********************]  2 of 2 completed

1 Failed download:
- AAPPPLL: No data found, symbol may be delisted


Adj Close               Close                High              \
             AAPPPLL         SPY AAPPPLL         SPY AAPPPLL         SPY   
Date                                                                       
2022-02-11       NaN  433.428314     NaN  440.459991     NaN  451.609985   
2022-02-14       NaN  432.011322     NaN  439.019989     NaN  441.600006   
2022-02-15       NaN  438.978333     NaN  446.100006     NaN  446.279999   
2022-02-16       NaN  439.470337     NaN  446.600006     NaN  448.059998   
2022-02-17       NaN  430.082642     NaN  437.059998     NaN  446.570007   
...              ...         ...     ...         ...     ...         ...   
2023-02-06       NaN  409.829987     NaN  409.829987     NaN  411.290009   
2023-02-07       NaN  415.190002     NaN  415.190002     NaN  416.489990   
2023-02-08       NaN  410.649994     NaN  410.649994     NaN  414.529999   
2023-02-09       NaN  407.089996     NaN  407.089996     NaN  414.570007   
2023-02-10       NaN  408.040009     NaN  408.040009     NaN  408.440002   

               Low                Open              Volume             
           AAPPPLL         SPY AAPPPLL         SPY AAPPPLL        SPY  
Date                                                                   
2022-02-11     NaN  438.940002     NaN  449.410004     NaN  153214600  
2022-02-14     NaN  435.339996     NaN  439.920013     NaN  123006300  
2022-02-15     NaN  443.179993     NaN  443.730011     NaN   88482700  
2022-02-16     NaN  441.940002     NaN  443.929993     NaN   84863600  
2022-02-17     NaN  436.420013     NaN  443.220001     NaN  102259100  
...            ...         ...     ...         ...     ...        ...  
2023-02-06     NaN  408.100006     NaN  409.790009     NaN   60295300  
2023-02-07     NaN  407.570007     NaN  408.869995     NaN   90990700  
2023-02-08     NaN  409.929993     NaN  413.130005     NaN   76227500  
2023-02-09     NaN  405.809998     NaN  414.410004     NaN   78694900  
2023-02-10     NaN  405.010010     NaN  405.859985     NaN   70738000  

[251 rows x 12 columns]

In [198]:
class stock_data:
    import yfinance as yf
    import pandas as pd
    import matplotlib.pyplot as plt
    import numpy as np
    import scipy.stats
    
    def __init__(self, ticker, start_date, end_date, risk_free_rate=0):
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
        self.risk_free_rate = risk_free_rate
        self.data = yf.download(self.ticker, self.start_date, self.end_date)
        self.data['Daily Return'] = self.data['Adj Close'].pct_change(1)
        self.sortino_ratio = None
        self.sharpe_ratio = None
        self.probabilistic_sharpe_ratio = None
        
        
    def info(self):
        print(f'Ticker: {self.ticker}, start_date: {self.start_date}, end_date:{self.end_date}')
    
    def _compute_sharpe_ratio(self):
        mean_return = self.data.dropna()['Daily Return'].mean()
        std = self.data.dropna()['Daily Return'].std()
        self.sharpe_ratio = (mean_return-self.risk_free_rate) / std
        del mean_return
        del std
    
        
    def get_sharpe_ratio(self, daily=True):
        if not self.sharpe_ratio:
            self._compute_sharpe_ratio()
        if daily:
            return self.sharpe_ratio
        else:
            return self.sharpe_ratio * (252 ** 0.5)
    
    def _compute_sortino_ratio(self, target=0):
        df = self.data.dropna()
        mean_return = df['Daily Return'].mean()
        downside = df[df['Daily Return'] < target]['Daily Return']
        std = downside.std()
        self.sortino_ratio = (mean_return-self.risk_free_rate) / std
        del df      
        
    def get_sortino_ratio(self, daily=True):
        if not self.sortino_ratio:
            self._compute_sortino_ratio()
        if daily:
            return self.sortino_ratio
        else:
            return self.sortino_ratio * (252 ** 0.5)
    
    def _compute_probabilistic_sharpe_ratio(self, benchmark=0):
        if not self.sortino_ratio:
            self._compute_sortino_ratio()
        import scipy.stats
        skew = scipy.stats.skew(self.data["Daily Return"].dropna())
        # Use fisher kurtosis
        kurtosis = scipy.stats.kurtosis(self.data["Daily Return"].dropna(), fisher=True)  
        n = len(self.data)
        std = ( (1 / (n-1)) * (1 + 0.5 * self.sortino_ratio**2 - skew * self.sortino_ratio + (kurtosis / 4) * self.sortino_ratio**2))**0.5
        ratio = (self.sortino_ratio - benchmark) / std
        self.probabilistic_sharpe_ratio = scipy.stats.norm.cdf(ratio)

    def get_probabalistic_sharpe_ratio(self, daily=True):
        if not self.probabilistic_sharpe_ratio:
            self._compute_probabilistic_sharpe_ratio()
        if daily:
            return self.probabilistic_sharpe_ratio
        else:
            return self.probabilistic_sharpe_ratio * (252 ** 0.5)

    
    def plot_daily_returns(self, kind='line', alpha=None):
        if kind == 'line':
            self.data.dropna()['Daily Return'].plot(kind=kind, label=self.ticker)
            plt.legend()
        elif kind =='hist':
            self.data.dropna()['Daily Return'].plot(kind=kind, bins=100, label=self.ticker, alpha=alpha)
            plt.legend()
        else:
            raise TypeError(f'Plot Kind of {kind} is not supported. Only line and hist kinds are supported')
    
    def plot_cumulative_daily_return_rate(self):
        cumul_return = (1 + self.data['Daily Return'].dropna()).cumprod() - 1
        (cumul_return * 100).plot()
        plt.ylabel("Cumulative Return as %")
            
class stock_comp:
    import numpy as np
    def __init__(self, tickers, start_date, end_date):
        self.tickers = tickers
        self.ticker_dict = {}
        
        for ticker in tickers:
            stock_data_obj_ = stock_data(ticker, start_date, end_date)
            if len(stock_data_obj_.data) != 0:
                self.ticker_dict[ticker] = stock_data_obj_
            else:
                print(f'Unable to retrieve stock data for ticker: {ticker}. Excluding this ticker from analysis')
                self.tickers.remove(ticker)

        
        self.n = len(self.tickers)
        self.equal_weights = [(1 / self.n)] * self.n
        self.rets = pd.concat([df.data['Daily Return'].dropna() for df in self.ticker_dict.values()], axis=1)
        self.rets.columns = [ticker for ticker in self.ticker_dict.keys()]
        
        
    def get_ticker_dict(self):
        return self.ticker_dict
    
    def get_daily_returns(self):
        return self.rets
    
    def _gen_random_weights(self):
        import numpy as np
        weights = np.random.random(self.n)
        return weights/ np.sum(weights)
        
    def plot_daily_returns(self, kind='line', alpha=None):
        if kind == 'line':
            self.rets.plot(kind=kind)
        elif kind =='hist':
            self.rets.plot(kind=kind, bins=100, alpha=alpha)
            plt.legend()
        else:
            raise TypeError(f'Plot Kind of {kind} is not supported. Only line and hist kinds are supported')

            
        

In [155]:
my_stock = stock('COST', '2017-01-01', '2021-01-01')

[*********************100%***********************]  1 of 1 completed


In [199]:
tickers = ['AAPL', 'COST', 'TSLA', 'CAKE']
start_date = '2017-01-01'
end_date = '2021-01-01'

comp = stock_comp(tickers, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
